In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [24]:
pokemon = pd.DataFrame(columns = ["number", "name", "type", "ability", "hidden_ability", 
                                 "gender_ratio", "hatch_cycle", "Base_Experience_Yield", "height", "weight",
                                 "leveling_rate", "color", "base_friendship"], index = range(1, 900))
pokemon

,number,name,type,ability,hidden_ability,gender_ratio,hatch_cycle,Base_Experience_Yield,height,weight,leveling_rate,color,base_friendship
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Creating a function for scraping pokemon data

def scrap_pokemon_details(pokemon_name):
    """
    scrape details of pokemon from bulbapedia

    Args :
        pokemon_name(str) name of the pokemon, eg : Greninja 

    Returns:
        A dictionary will all the details 
    """
    
    try: 
        # Give URL 
        url = f"https://bulbapedia.bulbagarden.net/wiki/{pokemon_name}_(Pok%C3%A9mon)"
        
        # fetching the webpage 
        response = requests.get(url)
        response.raise_for_status()
        text = response.text
        
        # Creating a BeautifulSoup instance 
        soup = BeautifulSoup(text, "lxml")
        
        # Overall data 
        data = soup.find_all("td", class_ = "roundy")
        
        if len(data) < 40 :
            print(f"Don't have enought data for {pokemon_data}")
            return None
        
        # name, number, category
        basic = data[0]
        name = basic.find(["big", "b"]).text
        category = basic.find(["span"]).text
        number = soup.find(["th","span", "big"], class_ = "roundy").text.replace("\n", "")
        
        # Type 
        type_data = data[1]
        type_data_2 =  type_data.find_all(["span", "a"])
        type_ = type_data_2[2].text, type_data_2[4].text
        
        # ability and hidden ability
        ability_data = data[2]
        ability_data2 = ability_data.find_all(["a", "span"])
        clear_ability = []
        junk_text = ["Cacophony", "Abilities", "Gen IV+", "Battle Bond", "Battle Bond Greninja"]

        for i in ability_data2:
            text = i.text.strip()
            if text and text not in junk_text:
                clear_ability.append(text)
        
        if len(clear_ability) == 6:
            ability = clear_ability[0], clear_ability[2]
            hidden_ability = clear_ability[4]
        elif len(clear_ability) == 4:
            ability = clear_ability[0]
            hidden_ability = clear_ability[2]
        elif len(clear_ability) == 2:
            ability = clear_ability[0]
            hidden_ability = None
        else:
            ability = None
            hidden_ability = None
            
        # gender ratio
        gender_data = data[3]
        gender_data_2 = gender_data.find_all(["a"])
        gender_ratio = gender_data_2[1].text
        
        # Catch rate 
        catch_rate = data[4].find("small").text
        
        # egg group and hatch cycle 
        egg_group_data = data[5]
        egg_group = egg_group_data.find("table", class_ = "roundy").text.strip()
        hatch_cycle_data = data[5].find_all("table", class_ = "roundy")
        hatch_cycle = hatch_cycle_data[1].text.strip().replace("\xa0", "")
        
        # Height and Weight 
        height_data = data[6].find_all("tr")
        height_data_2 = height_data[0].find_all("td")
        height = height_data_2[1].text.replace("\n", "")
        weight_data = data[7].find_all("tr")
        weight_data_2 = weight_data[0].find_all("td")
        weight = weight_data_2[1].text.replace("\n", "")
        
        # Base Experience Yield 
        bey_data = data[9].find("tr")
        bey_data_2 = bey_data.find_all("td")
        bey = bey_data_2[2].text.replace("V+", "").replace("\n", "")
        
        # Other details 
        leveling_rate = data[10].tr.text.strip()
        color = data[38].td.text.strip()
        base_friendship = data[39].tr.text.strip()
        
        return {"number" : number, "name" : name, "type" : type_, "ability" : ability, "hidden_ability" : hidden_ability, 
                 "gender_ratio" : gender_ratio, "hatch_cycle" : hatch_cycle, "Base_Experience_Yield" : bey, "leveling_rate" : leveling_rate,
                 "color" : color, "base_friendship" : base_friendship, "height" : height, "weight" : weight}
            
    except requests.RequestException as e:
        print(f"Failed to fetch data for {pokemon_name}: {e}")
        return None
    except (AttributeError, IndexError) as e:
        print(f"Failed to parse data for {pokemon_name}: {e}")
        return None

            

In [25]:
# Bulbasaur evolution line
bulbasaur_data = scrap_pokemon_details("Bulbasaur")
pokemon.loc[1] = bulbasaur_data
ivysaur_data = scrap_pokemon_details("Ivysaur")
pokemon.loc[2] = ivysaur_data
venusaur_data = scrap_pokemon_details("Venusaur")
pokemon.loc[3] = venusaur_data

In [27]:
# Charmander line 
charmander_data = scrap_pokemon_details("Charmander")
pokemon.loc[4] = charmander_data
charmeleon_data = scrap_pokemon_details("Charmeleon")
pokemon.loc[5] = charmeleon_data
charizard_data = scrap_pokemon_details("Charizard")
charizard_data["color"] = "Red"
pokemon.loc[6] = charizard_data

In [260]:
# Squirtle line 
squirtle_data = scrap_pokemon_details("Squirtle")
pokemon.loc[7] = squirtle_data
wartortle_data = scrap_pokemon_details("Wartortle")
pokemon.loc[8] = wartortle_data
blastoise_data = scrap_pokemon_details("Blastoise")
pokemon.loc[9] = blastoise_data

In [261]:
# Caterpie line 
caterpie_data = scrap_pokemon_details("Caterpie")
pokemon.loc[10] = caterpie_data
metapod_data = scrap_pokemon_details("Metapod")
pokemon.loc[11] = metapod_data
butterfree_data = scrap_pokemon_details("Butterfree")
pokemon.loc[12] = butterfree_data

In [262]:
# Weedle line 
weedle_data = scrap_pokemon_details("Weedle")
pokemon.loc[13] = weedle_data
kakuna_data = scrap_pokemon_details("Kakuna")
pokemon.loc[14] = kakuna_data
beedrill_data = scrap_pokemon_details("Beedrill")
pokemon.loc[15] = beedrill_data

In [263]:
#pidgey line
pidgey_data = scrap_pokemon_details("Pidgey")
pokemon.loc[16] = pidgey_data
pidgeotto_data = scrap_pokemon_details("Pidgeotto")
pokemon.loc[17] = pidgeotto_data
pidgeot_data = scrap_pokemon_details("Pidgeot")
pokemon.loc[18] = pidgeot_data

In [28]:
pokemon[0 : 18]

,number,name,type,ability,hidden_ability,gender_ratio,hatch_cycle,Base_Experience_Yield,height,weight,leveling_rate,color,base_friendship
1,#0001,Bulbasaur,"(Grass, Poison)",Overgrow,Chlorophyll,"87.5% male, 12.5% female",20cycles,64,0.7 m,6.9 kg,Medium Slow,Green,70
2,#0002,Ivysaur,"(Grass, Poison)",Overgrow,Chlorophyll,"87.5% male, 12.5% female",20cycles,142,1.0 m,13.0 kg,Medium Slow,Green,70
3,#0003,Venusaur,"(Grass, Poison)","(Overgrow, Chlorophyll)",Thick Fat,"87.5% male, 12.5% female",20cycles,236,2.0 m,100.0 kg,Medium Slow,Green,70
4,#0004,Charmander,"(Fire, Unknown)",Blaze,Solar Power,"87.5% male, 12.5% female",20cycles,62,0.6 m,8.5 kg,Medium Slow,Red,70
5,#0005,Charmeleon,"(Fire, Unknown)",Blaze,Solar Power,"87.5% male, 12.5% female",20cycles,142,1.1 m,19.0 kg,Medium Slow,Red,70
6,#0006,Charizard,"(Fire, Flying)","(Blaze, Solar Power)",Tough Claws,"87.5% male, 12.5% female",20cycles,240,1.7 m,90.5 kg,Medium Slow,Red,70
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
text = requests.get("https://bulbapedia.bulbagarden.net/wiki/Greninja_(Pok%C3%A9mon)").text
soup = BeautifulSoup(text, "lxml")
data = soup.find_all("td", class_ = "roundy")

In [21]:
# ability and hidden ability
ability_data = data[2]
ability_data2 = ability_data.find_all(["a", "span"])
clear_ability = []
junk_text = ["Cacophony", "Abilities", "Gen IV+", "Battle Bond", "Battle Bond Greninja"]

for i in ability_data2:
    text = i.text.strip()
    if text and text not in junk_text:
        clear_ability.append(text)
        
if len(clear_ability) > 5:
    ability = clear_ability[0], clear_ability[2]
    hidden_ability = clear_ability[4]
elif len(clear_ability) < 5:
    ability = clear_ability[0]
    hidden_ability = clear_ability[2]
print(len(clear_ability), ability, hidden_ability)
clear_ability

4 Torrent Protean


['Torrent', 'Torrent', 'Protean', 'Protean']